In [ ]:
pip install python-dotenv

In [ ]:
!pip install openai
# !pip install openai==0.28

In [103]:
import os
from dotenv import load_dotenv
import openai
import json
import datetime

In [104]:
# env_path = os.path.join(os.path.dirname(__file__), '../.env')
# load_dotenv(env_path)
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [105]:
import pandas as pd
pd.get_option("display.max_columns")
pd.get_option("display.max_colwidth")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

# load CSV to dataframe

In [106]:
df_csv = pd.read_csv('./data/Jobs_NYC_Postings.csv', header=0)
df_csv[0:1]

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Career Level,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,606346,DEPARTMENT FOR THE AGING,External,1,Business Operations Analyst,ASSOCIATE STAFF ANALYST,Competitive-1,12627,00,"Administration & Human Resources Technology, Data & Innovation Policy, Research & Analysis",F,Experienced (non-manager),70611.0,81203.0,Annual,"2 Lafayette St., N.Y.",Human Resources,The Office of Human Resources (OHR) seeks a highly motivated individual to serve as a Business O...,"1. A masterâs degree from an accredited college or university, accredited by regional, nationa...",â¢ Project management experience and expertise is a plus. â¢ Self-starter; works independently...,NaN,Please be sure to submit a resume & cover letter when applying. All current City Employees may a...,NaN,NaN,NaN,"New York City residency is generally required within 90 days of appointment. However, City Emplo...",09/28/2023,27-DEC-2023,09/29/2023,11/20/2023


In [107]:
df_jobs = df_csv.loc[0:1, ["Job ID", "Business Title", "Job Category", "Minimum Qual Requirements", "Preferred Skills"]]
df_jobs

,Job ID,Business Title,Job Category,Minimum Qual Requirements,Preferred Skills
0,606346,Business Operations Analyst,"Administration & Human Resources Technology, Data & Innovation Policy, Research & Analysis","1. A masterâs degree from an accredited college or university, accredited by regional, nationa...",â¢ Project management experience and expertise is a plus. â¢ Self-starter; works independently...
1,571361,Executive Program Manager,"Communications & Intergovernmental Affairs Engineering, Architecture, & Planning",At least six years of full-time satisfactory experience in construction management work on capit...,"â¢\tPrior project management, construction management and/or engineering coursework or experien..."


# process with OpenAI API

In [108]:
# model_engine = "gpt-3.5-turbo"
model_engine = "gpt-4"
openai.api_key = os.environ['OPENAI_API_KEY']

d = datetime.datetime.now()
f = '%Y%m%d%H%M%S'
strdatetime = d.strftime(f)
file_name = ".\log\openai_api_" + strdatetime + ".json"
open(file_name, "w") 

<_io.TextIOWrapper name='.\\log\\openai_api_20231210000533.json' mode='w' encoding='utf-8'>

In [109]:
def create_system_message(job_title, job_category, minimum_qual_requirements, preferred_skills):
    message = """
    You are an experienced job search agent.

    Instructions:
    - Please answer the most critical requirements including skills and qualifications for the given job post.
    - First of all, please consider the reason why thre requirement is critical. After that, please answer the requirement.

    - Answer in the Json format below:
    {"reason":"<reason>", "requirement":"<requirement>"}
    {"reason":"<reason>", "requirement":"<requirement>"}
    {"reason":"<reason>", "requirement":"<requirement>"}
    """
    prompt = {"role": "system", "content": message}
    return prompt

In [110]:
def create_user_message(job_title, job_category, minimum_qual_requirements, preferred_skills):
    message = f"""
    Please answer the most critical requirements including skills and qualifications for the given job post.

    [job post]
    job_title: {job_title}
    job_category: {job_category}
    minimum_qual_requirements: {minimum_qual_requirements}
    preferred_skills: {preferred_skills}
    """
    prompt = {"role": "user", "content": message}
    return prompt

In [111]:
def create_fewshot_assistant_message():
    message = """{"reason":"This job deals with various aspects of business operation, thus requiring a high-level of expertise.", "requirement":"A master's degree from an accredited college or university in related fields such as economics, finance, accounting, business or public administration, or human resources management."}
    {"reason":"The tasks described in the job post require experience to be carried out effectively and efficiently.", "requirement":"One year of satisfactory full-time professional experience in related areas such as budget administration, accounting, economic or financial administration, or fiscal or economic research."}
    {"reason":"The nature of this job requires excellent project management and organizational skills to manage multiple tasks simultaneously and ensure smooth operation.", "requirement":"Project management experience and excellent organizational skills."}
    {"reason":"Effective communication is important in this job role to ensure clear conveyance of plans and strategies to team members.", "requirement":"Excellent oral and written communication skills."}
    {"reason":"The job involves working with multiple technology tools to streamline tasks and requires proficiency in using them.", "requirement":"Highly proficient with Microsoft Office and the ability to seamlessly use technology to execute/streamline tasks."}"""
    assistant_response = {"role": "assistant", "content": message}
    return assistant_response

In [112]:
# df_jobs.loc[0:0,]
# ff means "for fewshot"
for index_ff, row_ff in df_jobs[0:1].iterrows():
    job_title_ff = row_ff["Business Title"]
    job_category_ff = row_ff["Job Category"]
    job_requirements_ff = row_ff["Minimum Qual Requirements"]
    preferred_skills_ff = row_ff["Preferred Skills"]

In [113]:
df_response = pd.DataFrame(columns=["jobid", "reason", "requirement"])
i=0

for index, row in df_jobs[0:].iterrows():

    prompt_message = []
    
    job_id = row["Job ID"]
    job_title = row["Business Title"]
    job_category = row["Job Category"]
    job_requirements = row["Minimum Qual Requirements"]
    preferred_skills = row["Preferred Skills"]

    print("job_id: " + str(job_id))

    system_message = create_system_message(job_title, job_category, job_requirements, preferred_skills)
    print (system_message)
    prompt_message.append(system_message)

    ############### fewshot ################
    user_message = create_user_message(job_title_ff, job_category_ff, job_requirements_ff, preferred_skills_ff)
    print (user_message)
    prompt_message.append(user_message)

    assistant_message = create_fewshot_assistant_message()
    print (assistant_message)
    prompt_message.append(assistant_message)
    ############### fewshot ################

    user_message = create_user_message(job_title, job_category, job_requirements, preferred_skills)
    print (user_message)
    prompt_message.append(user_message)

    print (prompt_message)

    response = openai.ChatCompletion.create(
        model=model_engine,
        messages=prompt_message,
        # temperature=1.0,
        # max_tokens=100,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0.6,
        # stop=["\n", "###"]
    )
    chat_responses = response["choices"][0]["message"]["content"].strip()
    print("chat_responses: " + chat_responses)

    answers = chat_responses.splitlines()
    for answer in answers:
        if answer == "":
            continue
        print("answer" + "[" + str(i) + "]: " + answer)
        answer_json = json.loads(answer)
        reason = answer_json["reason"]
        requirement = answer_json["requirement"]
        df_response.loc[i] = [job_id, reason, requirement]
        i+=1
    

    response_message = {
        "job_id": job_id,
        "job_title": job_title,
        "job_category": job_category,
        "job_requirements": job_requirements,
        "preferred_skills": preferred_skills,
        "prompt": prompt_message,
        "response": response,
    }
    
    with open(file_name, "a") as f:
        json.dump(response_message, f, indent=4)
        f.write(",\n")

job_id: 606346
{'role': 'system', 'content': '\n    You are an experienced job search agent.\n\n    Instructions:\n    - Please answer the most critical requirements including skills and qualifications for the given job post.\n    - First of all, please consider the reason why thre requirement is critical. After that, please answer the requirement.\n\n    - Answer in the Json format below:\n    {"reason":"<reason>", "requirement":"<requirement>"}\n    {"reason":"<reason>", "requirement":"<requirement>"}\n    {"reason":"<reason>", "requirement":"<requirement>"}\n    '}
{'role': 'user', 'content': '\n    Please answer the most critical requirements including skills and qualifications for the given job post.\n\n    [job post]\n    job_title: Business Operations Analyst\n    job_category: Administration & Human Resources Technology, Data & Innovation Policy, Research & Analysis\n    minimum_qual_requirements: 1. A masterâ\x80\x99s degree from an accredited college or university, accredited

In [114]:
df_response.loc[0:, ["requirement", "reason"]]

,requirement,reason
0,"A master's degree from an accredited college or university in related fields such as economics, ...","This job deals with various aspects of business operation, thus requiring a high-level of expert..."
1,One year of satisfactory full-time professional experience in related areas such as budget admin...,The tasks described in the job post require experience to be carried out effectively and efficie...
2,Project management experience and demonstrated organizational skills.,The nature of this job requires excellent project management and organizational skills to manage...
3,Excellent oral and written communication skills.,Effective communication is crucial in this job role since it aids in the clear conveyance of pla...
4,Proficiency with Microsoft Office and the ability to seamlessly use technology to execute/stream...,The job involves working with multiple technological tools to streamline tasks and requires prof...
5,At least six years of full-time satisfactory experience in construction management work on capit...,"The job demands handling of capital construction projects with large budgets, hence, prior exper..."
6,At least eighteen months of experience in a managerial or consultative capacity in an organizati...,Managerial expertise is critical in this role which includes overseeing teams and projects.
7,"A bachelor's degree in architecture, landscape architecture, or related engineering fields or a ...",The job requires technical knowledge and proficiency specific to construction or architecture.
8,Excellent oral and written communication skills towards a technical and non-technical audience.,Executive Program Manager is expected to coordinate with multiple teams and stakeholders making ...
9,Business process analysis skills with strong ability to understand and assess existing processes...,"The job requires the ability to examine, understand, and enhance existing work processes."


In [115]:
df_response.to_csv("./data/df_response.csv", index=False)